# Traveling Salesman Problem -- Tabu Search

Given a list of cities and the distances bewteen each pair of them, find the shortest less expensive path that visits each city exactly once such that it returns to the origin. 

## Constaints and considerations:

1. Label each city as 0, 1, 2, ..., N-1
2. The Salesman departs from the city n0 always.
3. Going from A to B has the same cost as departing from B to A.
4. A solution of the problem is a permutation of the N cities. 
5. Initial solution must be generated using a greedy algorithm. Once taking n0, the N-1 cities left are reviewed and the cheapest path is chosen. This process will be repeated with the remaining cities. IMPORTANT: as we are generating permutations, no city will be repeated in the solution. 
6. Neighborhoods are generated by choosing randomly a permutation position. The N-2 new solutions are generated by moving the city in that position to any other of the N-2 remaining possible postions. 
7. The Tabu List saves the city used to generate the neighborhood and will have a tabu tenure of floor(N/2). 

## Input:
1. First line will contain a number N of cities.
2. Second line will have the maximum number of iterations Imax of the Tabu Search algorithm.
3. The next N-1 lines will indicate the cost of going from one city to another: 
**Example**

10

100

49 30 53 72 19 76 87 45 48

19 38 32 31 75 69 61 25

41 98 56 6  6  45 53

52 29 46 90 23 98

63 90 69 50 82

60 88 41 95

61 92 10

82 73

5


## Output:
1. A permuation of the trayectory that the salesperson must follow
2. Cost of the path found.
**Example**

0 5 2 7 8 4 1 3 9 6

467

In [1]:
A = [[49, 30, 53, 72, 19, 76, 87, 45, 48],
     [19, 38, 32, 31, 75, 69, 61, 25],
     [41, 98, 56, 6, 6, 45, 53],
     [52, 29, 46, 90, 23, 98],
     [63, 90, 69, 50, 82],
     [60, 88, 41, 95],
     [61, 92, 10],
     [82, 73],
     [5]]

In [50]:
import numpy as np
import math
import statistics

First, I'm creating a dictionary to represent the 2D-array so I can iterate easily over the distances so I can get the initial solution.

The representation will return a dictionary with labels of the N cities and a Simetric matrix of the distances. 

In [3]:
def representacion(N,C):
    
    #Getting an upper triangular matrix
    for i in range(len(C)):
        C[i].insert(0,0)
    C.append([0])
    
    for i in C:
        if len(i)!=10:
            while len(i)!=10:
                i.insert(0,0)
                
    #Getting a simmetric matrix
    B = np.array(C).transpose()
    C = np.array(C)
    D = B+C
    
    dic = {} 
    for i in range(N):
        dic[i] = C[i]
    dic[N] = [0]
    return dic, D

In [4]:
prueba,pesos = representacion(9,A)

In [5]:
prueba, pesos

({0: array([ 0, 49, 30, 53, 72, 19, 76, 87, 45, 48]),
  1: array([ 0,  0, 19, 38, 32, 31, 75, 69, 61, 25]),
  2: array([ 0,  0,  0, 41, 98, 56,  6,  6, 45, 53]),
  3: array([ 0,  0,  0,  0, 52, 29, 46, 90, 23, 98]),
  4: array([ 0,  0,  0,  0,  0, 63, 90, 69, 50, 82]),
  5: array([ 0,  0,  0,  0,  0,  0, 60, 88, 41, 95]),
  6: array([ 0,  0,  0,  0,  0,  0,  0, 61, 92, 10]),
  7: array([ 0,  0,  0,  0,  0,  0,  0,  0, 82, 73]),
  8: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 5]),
  9: [0]},
 array([[ 0, 49, 30, 53, 72, 19, 76, 87, 45, 48],
        [49,  0, 19, 38, 32, 31, 75, 69, 61, 25],
        [30, 19,  0, 41, 98, 56,  6,  6, 45, 53],
        [53, 38, 41,  0, 52, 29, 46, 90, 23, 98],
        [72, 32, 98, 52,  0, 63, 90, 69, 50, 82],
        [19, 31, 56, 29, 63,  0, 60, 88, 41, 95],
        [76, 75,  6, 46, 90, 60,  0, 61, 92, 10],
        [87, 69,  6, 90, 69, 88, 61,  0, 82, 73],
        [45, 61, 45, 23, 50, 41, 92, 82,  0,  5],
        [48, 25, 53, 98, 82, 95, 10, 73,  5,  0]]))

Now I can think about generating an initial solution. 

In [6]:
from random import choice
from random import randint
def initial_solution(N,Dic,distances):
    #This is to know which cities are left to be visted:
    lista = [i for i in range(1,N)]
    
    #Starting from N0
    x = [0]
    
    distancia = min(Dic[x[0]][(x[0]+1):])#Smallest distance
    ciudad = list(Dic[x[0]])
    i = ciudad.index(distancia)
    lista.remove(i) #once the city has been visited, erase it 

    
    while lista:
        aux = i
        x.append(i)
        
        #Idea: take the smallest and check the smallest distance in the dictionary
        if i<=(N-2):
            if i in lista:
                distancia = min(Dic[i][(i+1):]) #Taking from one 'cause there is a 0 
                ciudad = list(Dic[i])
                i = ciudad.index(distancia) #Check the next city
                if i in lista: #Check if it is still available 
                    lista.remove(i)
                else:
                    i = choice(lista)
                    lista.remove(i)
                    
            if i not in lista and len(x)<=2:  #This is because X has things inside when the loop started
                if i<N-3:
                    distancia = sorted(Dic[i][i+1:])[1]
                    ciudad = list(Dic[i])
                    i = ciudad.index(distancia)
                else:
                    i = choice(lista)
            if i not in lista:
                i = choice(lista)
                lista.remove(i)
                
        else: #In case I'm iterating over the last elements of the "lista"
            if i in lista:
                distancia = min(list(distances[i][:i]))
                ciudad = list(distances[i])
                i = ciudad.index(distancia)
                if i in lista:
                    lista.remove(i)
                else:
                    i = choice(lista)
                    lista.remove(i)
            if i not in lista:
                distancia = list(sorted(list(distances[i][:i])))[1]
                ciudad = list(distances[i])
                
                if ciudad.index(distancia) in lista:
                    i = ciudad.index(distancia)
                    lista.remove(i)
                else:
                    i = choice(lista)
                    lista.remove(i)        
        if len(x)==N-1:
            x.append(i)

    return x

In [7]:
x0 = initial_solution(10,prueba,pesos)
x0

[0, 5, 6, 6, 8, 3, 2, 4, 1, 7]

In order to create the Search, we need a way of:
1. Create Neighbors
2. Check if it goes inside the tabu list
3. Create a reduced neighborhood
4. Check the sum of distances


In [8]:
def Neighborhood(x0): #Create neighborhood
    city = choice(x0) #This city is the one we are going to move along the list
    position = x0.index(city)
    aux1 = []
    N = []
    Ne = []
    
    #Eliminating the city to be moved
    for i in x0:
        if i!= city:
            aux1.append(i)
    
    #Auxiliary
    for i in range(len(x0)-1):
        N.append(aux1)
    
    #Moving along the list the city
    Ne.append([city]+N[0])
    control = 1
    for i in N:
        A = i[:control]
        b = [city]
        C = i[control:]
        lista = A+b+C
        Ne.append(lista)
        control +=1
    
    #Check where the permutation of x0 is and eliminating that from Neighborhood
    for i in Ne:
        if i==x0:
            Ne.remove(i)
    return position, Ne    
    

In [9]:
city, neigh = Neighborhood(x0)
city, neigh

(7,
 [[4, 0, 5, 6, 6, 8, 3, 2, 1, 7],
  [0, 4, 5, 6, 6, 8, 3, 2, 1, 7],
  [0, 5, 4, 6, 6, 8, 3, 2, 1, 7],
  [0, 5, 6, 4, 6, 8, 3, 2, 1, 7],
  [0, 5, 6, 6, 4, 8, 3, 2, 1, 7],
  [0, 5, 6, 6, 8, 4, 3, 2, 1, 7],
  [0, 5, 6, 6, 8, 3, 4, 2, 1, 7],
  [0, 5, 6, 6, 8, 3, 2, 1, 4, 7],
  [0, 5, 6, 6, 8, 3, 2, 1, 7, 4]])

In [10]:
x0

[0, 5, 6, 6, 8, 3, 2, 4, 1, 7]

In [11]:
def Sum_distances(x,pesos):
    suma = 0
    for i in range(len(x)):
        
        #Sum distances from the first to the last city
        if i<len(x)-1:
            city_now = x[i]
            city_next = x[i+1]
            suma += pesos[city_now][city_next]
            
        #Sum distnances from the las city to the first one
        else:
            city_now = x[i]
            city_next = x[0]
            suma += pesos[city_now][city_next]
    return suma

In [12]:
Sum_distances(x0,pesos)

521

In [13]:
def Aspirantes(N,number): #Random permutations of Solutions. 
    #Input, the N city names and the amount of aspirants we want
    M = []
    
    while len(M)!=number:
        lista = [i for i in range(N)]
        x = []
        for i in range(N):
            elem = choice(lista)
            x.append(elem)
            lista.remove(elem)
        
        #In case there are repetitions within the aspirants
        if x not in M: 
            M.append(x)
    return M

In [14]:
A = Aspirantes(10,10)
A

[[4, 2, 5, 9, 7, 1, 3, 6, 8, 0],
 [4, 9, 6, 2, 0, 3, 1, 7, 8, 5],
 [1, 5, 8, 4, 6, 0, 9, 3, 2, 7],
 [8, 9, 0, 5, 6, 1, 3, 4, 2, 7],
 [6, 1, 5, 4, 3, 2, 7, 0, 8, 9],
 [2, 0, 1, 4, 6, 5, 3, 7, 8, 9],
 [7, 0, 2, 9, 3, 8, 6, 5, 1, 4],
 [7, 8, 5, 6, 1, 9, 2, 0, 4, 3],
 [1, 4, 7, 6, 5, 3, 9, 8, 2, 0],
 [6, 5, 7, 0, 9, 4, 1, 2, 3, 8]]

In [31]:
def Reduced_N(Solution, Ne, Aspirants, T): #Create reduced neighborhood
    reduced = []
    A = []
    tabu = []
    
    for i in T:
        indice = Solution.index(i[0])
        B = Solution.copy()
        po = B.pop(indice)
        for j in range(len(Solution)):
            A.append(B[:j]+[i[0]]+B[j:])
    for i in A:
        if i not in Ne:
            reduced.append(i)
    
    reduced = reduced + Aspirants #Also used aspirants: in this case, random initialization
        
    return reduced



In [24]:
re  = Reduced_N(x0, neigh, A, [city])
re

[[7, 0, 5, 6, 6, 8, 3, 2, 4, 1],
 [0, 7, 5, 6, 6, 8, 3, 2, 4, 1],
 [0, 5, 7, 6, 6, 8, 3, 2, 4, 1],
 [0, 5, 6, 7, 6, 8, 3, 2, 4, 1],
 [0, 5, 6, 6, 7, 8, 3, 2, 4, 1],
 [0, 5, 6, 6, 8, 7, 3, 2, 4, 1],
 [0, 5, 6, 6, 8, 3, 7, 2, 4, 1],
 [0, 5, 6, 6, 8, 3, 2, 7, 4, 1],
 [0, 5, 6, 6, 8, 3, 2, 4, 7, 1],
 [0, 5, 6, 6, 8, 3, 2, 4, 1, 7],
 [4, 2, 5, 9, 7, 1, 3, 6, 8, 0],
 [4, 9, 6, 2, 0, 3, 1, 7, 8, 5],
 [1, 5, 8, 4, 6, 0, 9, 3, 2, 7],
 [8, 9, 0, 5, 6, 1, 3, 4, 2, 7],
 [6, 1, 5, 4, 3, 2, 7, 0, 8, 9],
 [2, 0, 1, 4, 6, 5, 3, 7, 8, 9],
 [7, 0, 2, 9, 3, 8, 6, 5, 1, 4],
 [7, 8, 5, 6, 1, 9, 2, 0, 4, 3],
 [1, 4, 7, 6, 5, 3, 9, 8, 2, 0],
 [6, 5, 7, 0, 9, 4, 1, 2, 3, 8]]

In [25]:
def BestNow(M,pesos):
    #Get the list of possible solitions and the adjacency matrix for the distances between cities
    scores = Sum_distances(M[0],pesos)
    lista_ganadora = 0
    
    #Calculating the sum of distance and keeping the best so far
    for i in M:
        now = Sum_distances(i,pesos)
        if scores>= now:
            scores = now
            lista_ganadora = i
            
    return scores, lista_ganadora #Returns the best score and the asociated trajectory

In [26]:
score, ganadora = BestNow(re,pesos)
score, ganadora

(391, [0, 5, 6, 6, 8, 3, 2, 7, 4, 1])

We also need a mecanism in order to update the tabu list. 

In [27]:
def Update_list(T,tenure):
    for i in range(len(T)):
        #Delete element of list if tenure time is completed
        if T[i][1]==tenure: 
            T.pop(i)
            
        #Add a time unit with each iteration
        else:
            list(T[i])[1] += 1
            tuple(T[i])
    return T

Now the actual search:

In [42]:
def Busqueda(N, A, Imax):
    Diccionario,pesos = representacion(N,A)
    x0 = initial_solution(N,Diccionario,pesos)

    lista_tabu = []
    tenure = math.floor(N/2)
    
    k = 0
    Best_x = x0 #global... allegedly
    Best_here = x0 #local
    
    Best_score = Sum_distances(Best_x,pesos)
    Best_local = Best_score
    
    while k<Imax:
        k+=1
        city, neigh = Neighborhood(Best_x)
        aspirantes = Aspirantes(N, N)
        lista_tabu.append((Best_here[city],1)) #inserted the city and the time to live
        reducido = Reduced_N(Best_here, neigh,aspirantes,lista_tabu)
        Best_local, Best_here = BestNow(reducido,pesos)
        
        if Best_local<=Best_score:
            Best_score = Best_local
            Best_x = Best_here
            
        lista_tabu = Update_list(lista_tabu,tenure) 
    return Best_x, Best_score

In [46]:
A = [[49, 30, 53, 72, 19, 76, 87, 45, 48],
     [19, 38, 32, 31, 75, 69, 61, 25],
     [41, 98, 56, 6, 6, 45, 53],
     [52, 29, 46, 90, 23, 98],
     [63, 90, 69, 50, 82],
     [60, 88, 41, 95],
     [61, 92, 10],
     [82, 73],
     [5]]

In [47]:
Busqueda(10,A,100)

([5, 0, 2, 7, 6, 9, 8, 3, 4, 1], 269)

# Second part

Allow the user to execute M times your algorithm of TB for the Taveling Salesman Problem. The program will be able to read a document with the input described at the beginning. After the M excutions, report:

1. Best solution found.
2. Wors solution found.
3. Solution corresponding to the median of the M executions.
4. Median of the target function considering the M executions.
5. Standar deviation of the target function considering the M executions.

In the first three points indicate the value of x (permutation) and target function value. 

*Important:  the user will indicate the name of the file of input and the numer of M executions to make.*

In [51]:
if __name__=='__main__':
    filename = str(input("Filename: "))
    M = int(input("M executions: "))

    def Read(filename):
        file1 = open(filename, 'r')
        Lines = file1.readlines()

        size = int(Lines[0]) #The number of cities
        Imax = int(Lines[1]) #Max iterations
        A = []
        for i in range(2,size+1): #skipping the first two lines read
            A.append((list(map(int,Lines[i].split()))))
        return size, A, Imax 


    Scores = {}
    Solution = {}
    for i in range(M):
        N,A,Imax = Read(filename)

        X,score = Busqueda(N, A, Imax)
        Scores[i] = score
        Solution[i] = X

    #This has to be first: point 1 and 3 need sorting the dictionary
    median_key = math.ceil(statistics.median(Scores.keys()))
    Median = [Solution[median_key],Scores[median_key]]
    mean_values = sum(Scores.values())/float(M)
    standard_deviation = np.std(np.array(list(Scores.values())))

    Scores = sorted(Scores.items(), key=lambda x: x[1])   
    Best_solutions = [Solution[Scores[0][0]],Scores[0][1]]
    Worst_solutions = [Solution[Scores[-1][0]],Scores[-1][1]]


    print("Best solution in ", M, " iterations: ", Best_solutions)
    print("Worst solution in ", M, " iterations: ", Worst_solutions)
    print("Solution of median in: ", M, " iterations: ", Median)
    print("Mean solution of target function: ", mean_values)
    print("Standard deviation of target function: ", standard_deviation)    

Filename: test.txt
M executions: 20
Best solution in  20  iterations:  [[0, 5, 3, 8, 9, 6, 2, 7, 4, 1], 248]
Worst solution in  20  iterations:  [[8, 9, 6, 3, 5, 0, 2, 7, 1, 4], 296]
Solution of median in:  20  iterations:  [[4, 1, 9, 6, 7, 2, 0, 5, 3, 8], 285]
Mean solution of target function:  272.15
Standard deviation of target function:  20.54574165125221
